In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
DATASET_PATH = "/kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/"

print(os.listdir(DATASET_PATH))
print(len(os.listdir(os.path.join(DATASET_PATH, "images"))))

# Any results you write to the current directory are saved as output.

['styles.csv', 'images.csv', 'images', 'styles']
44441


In [22]:
styles_df = pd.read_csv(os.path.join(DATASET_PATH, "styles.csv"), error_bad_lines=False)
styles_df.head()

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [23]:
images_df = pd.read_csv(os.path.join(DATASET_PATH, "images.csv"), error_bad_lines=False)
images_df.head()

,filename,link
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [24]:
styles_df['image'] = styles_df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
# drop unnecessary column productDisplayName
styles_df = styles_df.drop('productDisplayName', 1)
df = styles_df.drop('year', 1)
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,usage,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,Casual,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,Casual,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual,53759.jpg


In [26]:
import tensorflow as tf
batch_size = 32
learning_rate = 0.001

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# augmentation
image_generator = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.2,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=os.path.join(DATASET_PATH, "images"),
    x_col="image",
    y_col="masterCategory",
    target_size=(300,300),
    batch_size=batch_size,
    subset="training",
    class_mode='categorical'
)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=os.path.join(DATASET_PATH, "images"),
    x_col="image",
    y_col="masterCategory",
    target_size=(300,300),
    batch_size=batch_size,
    subset="validation",
    class_mode='categorical'
)

classes = len(training_generator.class_indices)
print("Number of classes ", classes)
class_mapping = {v: k for k, v in training_generator.class_indices.items()}
print(class_mapping)

/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 35536 validated image filenames belonging to 7 classes.
Found 8883 validated image filenames belonging to 7 classes.
Number of classes  7
{0: 'Accessories', 1: 'Apparel', 2: 'Footwear', 3: 'Free Items', 4: 'Home', 5: 'Personal Care', 6: 'Sporting Goods'}


In [38]:
model = None
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(classes, activation='softmax')
])


model.summary()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 33, 33, 128)      

In [39]:

class MyEarlyStopping(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
#     print(logs)
    if(logs.get('loss')<0.07):
      print("\nReached desired accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = MyEarlyStopping()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit_generator(training_generator, epochs=90, validation_data = validation_generator, verbose = 1, callbacks=[callbacks])

Epoch 1/90
 137/1111 [==>...........................] - ETA: 30:21 - loss: 1.5792 - acc: 0.4339